In [1]:
import pandas as pd

df_without_nan = pd.read_pickle('../data/training_classif_data.pkl')
df_with_nan = pd.read_pickle('../data/df_with_nan.pkl')


In [2]:
# Standardiser les données ????

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# Standardiser toutes les colonnes du DataFrame

# il faut encoder !! puis standardiser !!
# df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
# df.head()


In [3]:
# CLASSIFICATION 'Code_type_local'

# TRAINING

# Import train_test_split function
from sklearn.model_selection import train_test_split
# ########### Random forest
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Random forest mieux mais 8min de calcul


X = df_without_nan.drop(columns=['Code_type_local', 'Commune', 'Section', 'No_plan', 'Nature_culture',
                                 'Departement', 'Code_postal', 'day', 'month', 'year', 'latitude', 'longitude', 'Surface_terrain'])
                      

y = df_without_nan['Code_type_local']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1)


modele_decision_tree = DecisionTreeClassifier(random_state=42)

modele_decision_tree.fit(X_train, y_train)
y_pred = modele_decision_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9245617696160268


In [4]:
df_with_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360336 entries, 0 to 367818
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   No_disposition             360336 non-null  int64  
 1   Valeur_fonciere            360336 non-null  float64
 2   Code_postal                360336 non-null  object 
 3   Commune                    360336 non-null  object 
 4   Section                    360301 non-null  object 
 5   No_plan                    360336 non-null  int64  
 6   Nombre_de_lots             360336 non-null  int64  
 7   Code_type_local            234825 non-null  float64
 8   Surface_reelle_bati        234582 non-null  float64
 9   Nombre_pieces_principales  234582 non-null  float64
 10  Nature_culture             221055 non-null  object 
 11  Surface_terrain            221055 non-null  float64
 12  day                        360336 non-null  object 
 13  month                      36

In [5]:
# Still usefull variables with Nan values, let's remplace them with good values (median values from the good dataframe (df_without_nan))


# Match les valeurs de 'Nombre_pieces_principales' et de 'Surface_reelle_bati' avec les médianne où toutes les données sont bonnes

median_values = df_without_nan.groupby('Code_type_local')['Nombre_pieces_principales'].median()
print(median_values)
df_with_nan['Nombre_pieces_principales'] = df_with_nan['Code_type_local'].map(median_values)

# ################

median_values = df_without_nan.groupby('Code_type_local')['Surface_reelle_bati'].median()

print(median_values)
df_with_nan['Surface_reelle_bati'] = df_with_nan['Code_type_local'].map(median_values)


Code_type_local
1    4.0
2    3.0
3    0.0
4    0.0
Name: Nombre_pieces_principales, dtype: float64
Code_type_local
1     91.0
2     55.0
3      0.0
4    150.0
Name: Surface_reelle_bati, dtype: float64


In [6]:
# Filling Nans Columns

df_with_nan['Nombre_pieces_principales'] = df_with_nan['Nombre_pieces_principales'].fillna(df_with_nan['Nombre_pieces_principales'].median())
df_with_nan['Surface_reelle_bati'] = df_with_nan['Surface_reelle_bati'].fillna(df_with_nan['Surface_reelle_bati'].median())


In [7]:
# Use previous model to fill 'Code_type_local' == NaN

# Réimporter les données de base, tous ceux qui ont Code_type_local == NaN, puis appliquer le modèle le classif 

# Sélectionnez les individus avec Code_type_local manquant
individus_code_type_local_manquants = df_with_nan[df_with_nan['Code_type_local'].isna()]
#print(individus_code_type_local_manquants)

X_manquants = individus_code_type_local_manquants.drop(columns=['Code_type_local', 'Commune', 'Section', 'No_plan', 'Nature_culture',
                                                                'Departement', 'Code_postal', 'day', 'month', 'year', 'latitude', 'longitude', 'Surface_terrain'])
#print(X_manquants)


y = individus_code_type_local_manquants['Code_type_local']

# Prédisez Code_type_local pour les individus manquants
predictions_code_type_local = modele_decision_tree.predict(X_manquants)

# Remplacez les valeurs manquantes dans le DataFrame original avec les prédictions
df_with_nan.loc[df_with_nan['Code_type_local'].isna(), 'Code_type_local'] = predictions_code_type_local


print(len(df_with_nan))
df_with_nan.head()


360336


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year,latitude,longitude,Departement
0,1,195000.0,6000,NICE,LZ,31,2,2.0,55.0,3.0,NaN,NaN,27,3,2018,43.711999,7.238269,06
1,1,3000.0,3120,SAINT CHRISAINTOPHE,D,552,0,1.0,55.0,3.0,P,1911.0,8,6,2018,46.166316,3.578008,03
2,1,150000.0,3440,SAINT HILAIRE,AC,67,0,2.0,55.0,3.0,S,231.0,1,3,2018,46.486689,3.015434,03
3,1,210500.0,1400,CHANOZ CHATENAY,B,386,0,1.0,55.0,3.0,J,315.0,20,2,2018,46.186181,5.028077,01
4,1,165000.0,1210,FERNEY VOLTAIRE,AK,223,1,3.0,0.0,0.0,NaN,NaN,15,10,2018,46.251979,6.108264,01


In [8]:
df = df_with_nan

In [9]:
# Removing useless variables
df = df.drop(columns=["Nature_culture", "Surface_terrain"])

In [10]:
pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)
print(len(df))

df.head()

No_disposition               0.000000
Valeur_fonciere              0.000000
Code_postal                  0.000000
Commune                      0.000000
Section                      0.009713
No_plan                      0.000000
Nombre_de_lots               0.000000
Code_type_local              0.000000
Surface_reelle_bati          0.000000
Nombre_pieces_principales    0.000000
day                          0.000000
month                        0.000000
year                         0.000000
latitude                     0.000000
longitude                    0.000000
Departement                  0.000000
dtype: float64
360336


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,day,month,year,latitude,longitude,Departement
0,1,195000.0,6000,NICE,LZ,31,2,2.0,55.0,3.0,27,3,2018,43.711999,7.238269,06
1,1,3000.0,3120,SAINT CHRISAINTOPHE,D,552,0,1.0,55.0,3.0,8,6,2018,46.166316,3.578008,03
2,1,150000.0,3440,SAINT HILAIRE,AC,67,0,2.0,55.0,3.0,1,3,2018,46.486689,3.015434,03
3,1,210500.0,1400,CHANOZ CHATENAY,B,386,0,1.0,55.0,3.0,20,2,2018,46.186181,5.028077,01
4,1,165000.0,1210,FERNEY VOLTAIRE,AK,223,1,3.0,0.0,0.0,15,10,2018,46.251979,6.108264,01


In [11]:
# Adding Variable 'Prix_moyen_m2'

import numpy as np

# Groupez les données par 'Commune' et calculez la somme de 'Surface_reel' et 'Valeur_fonciere' pour chaque commune
groupe_commune = df.groupby('Commune').agg(
    {'Surface_reelle_bati': 'sum', 'Valeur_fonciere': 'sum'})

# Calculez le prix moyen au mètre carré par commune
groupe_commune['Prix_moyen_m2'] = groupe_commune['Valeur_fonciere'] / \
    groupe_commune['Surface_reelle_bati']


# Fusionnez le DataFrame original avec le DataFrame groupe_commune sur la colonne 'Commune'
df = pd.merge(df, groupe_commune[['Prix_moyen_m2']],
              left_on='Commune', right_index=True, how='left')



# Remplace les inf par NaN
df = df.replace([np.inf, -np.inf], np.nan)
# Supprime les lignes avec NaN dans 'Prix_moyen_m2'
df = df.dropna(subset=['Prix_moyen_m2'], how='any')

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 360336 entries, 0 to 367818
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   No_disposition             360336 non-null  int64  
 1   Valeur_fonciere            360336 non-null  float64
 2   Code_postal                360336 non-null  int64  
 3   Commune                    360336 non-null  object 
 4   Section                    360301 non-null  object 
 5   No_plan                    360336 non-null  int64  
 6   Nombre_de_lots             360336 non-null  int64  
 7   Code_type_local            360336 non-null  float64
 8   Surface_reelle_bati        360336 non-null  float64
 9   Nombre_pieces_principales  360336 non-null  float64
 10  day                        360336 non-null  int64  
 11  month                      360336 non-null  int64  
 12  year                       360336 non-null  int64  
 13  latitude                   36

In [12]:
# Export final data

df.to_pickle('../data/final_data.pkl')
